<a href="https://colab.research.google.com/github/GiorgiaAuroraAdorni/ML-bachelor-course-assignments-sp23/blob/main/assignment%201/deliverable/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1
Student: Guillaume Baranzini

--- 
# IMPORTANT: all the submitted code should be in 2 cells
1) How you trained, evaluated and saved your model
2) How to load your model from a file, load the data and evaluate the model. Cell 2) should be running independently (even if cell 1 is not run)

In [68]:
# Import libraries
import pickle
import io
import requests
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# Load data
url = 'https://drive.switch.ch/index.php/s/TeDwnbYsBKRuJjv/download'
response = requests.get(url)
data = np.load(io.BytesIO(response.content))

# x is a Numpy array of shape (n_samples, n_features) with the inputs
x = data.f.x
# y is a Numpy array of shape (n_samples, ) with the targets
y = data.f.y

# T1

# Split data into train and test set for x and y
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.90, shuffle=True, random_state=2)

print('train_data shape:', x_train.shape)
print('test_data shape:', x_test.shape)  # shape of the test sets + shape of the train sets

# Prepare the input features
# Compact form
x_1 = x_train[:,0].reshape(-1,1)
x_2 = x_train[:,1].reshape(-1,1)
x_3 = np.sin(x_2)
x_4 = x_1 * x_2

ones_vector = np.ones(shape=(len(x_train),1))  # Vector of 1
X = np.hstack((ones_vector, x_1, x_2, x_3, x_4))         # The compact input

print('Input (X) shape:\t', X.shape)

# Train the model
model = LinearRegression(fit_intercept=True)
model.fit(x_train, y_train)

# Print optimal parameters
theta_hat = np.linalg.solve(a=X.T.dot(X), b=X.T.dot(y_train))
print('Optimal parameters:\t', theta_hat)
print("f(x, θ):", theta_hat[0], "+", theta_hat[1], "*x1 +", theta_hat[2], "*x2 +", theta_hat[3], "*sin(x2) +", theta_hat[4], "*x1*x2")

# Evaluate the model
train_predictions = model.predict(x_train)
train_performance = mean_squared_error(train_predictions, y_train)
test_predictions = model.predict(x_test)
test_mse = mean_squared_error(test_predictions, y_test)

print("Train performance: \t", train_performance)
print('Test MSE:', test_mse)

# Save the trained model to a pickle file
with open('linear_regression.pickle', 'wb') as file:
    pickle.dump(model, file)
    
##################################################################################
    
# T2 
import numpy as np
import pickle
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

# Train the model using SGDRegressor
model1 = SGDRegressor(loss='squared_error', penalty=None, max_iter=250000, learning_rate='constant', eta0=0.0003, alpha=0.01, random_state=0)
model1.fit(x_train, y_train)

# Evaluate the model
train_predictions = model1.predict(x_train)
train_performance = mean_squared_error(y_train, train_predictions)

test_predictions = model1.predict(x_test)
test_mse = mean_squared_error(y_test, test_predictions)

print("Train performance: \t", train_performance)
print('Test MSE:', test_mse)

# Save the trained model to a pickle file
with open('nonlinear_model.pickle', 'wb') as file:
    pickle.dump(model1, file)

    
###########################################################3

# T3
from keras.models import Sequential
from keras.layers import Dense

# Define the model
def create_model():
    model = Sequential()
    model.add(Dense(16, input_dim=2, activation='relu')) # input 
    model.add(Dense(16, activation='relu'))              # add a layer with 16 neurons
    model.add(Dense(16, activation='relu'))              # add a layer with 16 neurons
    model.add(Dense(16, activation='relu'))              # add a layer with 16 neurons
    model.add(Dense(1, activation='linear'))             # output
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
    return model

# Create the model
model2 = create_model()

# Train the model
history = model2.fit(x_train, y_train, epochs=126, batch_size=8, verbose=0, validation_split=0.1)

# Evaluate the model on test data
_, mse = model2.evaluate(x_test, y_test)
print('MSE: %.4f' % mse)

# Saving the model as a pickle file
with open('model_task_3.pickle', 'wb') as f:
    pickle.dump(model2, f)

train_data shape: (1800, 2)
test_data shape: (200, 2)
Input (X) shape:	 (1800, 5)
Optimal parameters:	 [ 1.24205736 -0.04503563 -0.56462773  0.47663575  0.04029328]
f(x, θ): 1.242057355919393 + -0.04503563207911075 *x1 + -0.5646277318069006 *x2 + 0.4766357540857314 *sin(x2) + 0.04029327643848045 *x1*x2
Train performance: 	 0.7497017385955663
Test MSE: 0.7452176115055922
Train performance: 	 0.7548650381392601
Test MSE: 0.7574280358599361
7/7 [==============================] - 0s 4ms/step - loss: 0.0218 - mean_squared_error: 0.0218
MSE: 0.0218


# Example on how to use baseline model:

In [69]:
# Import librairies 
import pickle
import joblib
import io
import requests
import numpy as np


def evaluate_predictions(y_true, y_pred):
    """
    Evaluates the mean squared error between the values in y_true and the values
    in y_pred.
    ### YOU CAN NOT EDIT THIS FUNCTION ###
    :param y_true: Numpy array, the true target values from the test set;
    :param y_pred: Numpy array, the values predicted by your model.
    :return: float, the mean squared error between the two arrays.
    """
    assert y_true.shape == y_pred.shape
    return ((y_true - y_pred) ** 2).mean()


def load_model(filename):
    """
    Loads a Scikit-learn model saved with joblib.dump.
    This is just an example, you can write your own function to load the model.
    Some examples can be found in src/utils.py.
    :param filename: string, path to the file storing the model.
    :return: the model.
    """
    model = joblib.load(filename)

    return model

# Load the data
# This will be replaced with our private test data when grading the assignment

# Load data from url
url = 'https://drive.switch.ch/index.php/s/TeDwnbYsBKRuJjv/download'
response = requests.get(url)
data = np.load(io.BytesIO(response.content))

# Alternatively yo can load the data from file
# data_path = '../data/data.npz'
# data = np.load(data_path)

# x is a Numpy array of shape (n_samples, n_features) with the inputs
x = data.f.x
# y is a Numpy array of shape (n_samples, ) with the targets
y = data.f.y

# Load the trained model

# for Task 1
#baseline_model_path = './linear_regression.pickle' 

# for Task 2
#baseline_model_path = './nonlinear_model.pickle'

# for task 3
baseline_model_path = './model_task_3.pickle'

baseline_model = load_model(baseline_model_path)

# Change input
x = data.f.x

# Predict on the given samples
 
y_pred = baseline_model.predict(x).flatten()


############################################################################
# STOP EDITABLE SECTION: do not modify anything below this point.
############################################################################

# Evaluate the prediction using MSE
mse = evaluate_predictions(y_pred, y)
print(f'MSE on whole dataset: {mse}')

# NOTE: NOW THIS CELL IS NOT WORKING SINCE YOU NEED TO CHANGE THE INPUT.
# DO IT AND EVERYTHING RUNS SMOOTH 

63/63 [==============================] - 0s 1ms/step
MSE on whole dataset: 0.017099708026528618
